In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.io

In [ ]:
#load the data
single_subj = scipy.io.loadmat('/Users/yat-lok/')


1. data:graph data for clinical groups
2. model: graph convolution network? how to convert my graph data to vectors?
3. loss: 
4. outcome: prediction, if this graph is belong to different clinical groups.